#   Exercise 4 - Computer Vision

##  Task 3- Object Detection

Before you begin this tutorial you have to complete the following tasks explained in the handout
- [x] Setting up the gazebo simulation as a ROS package
- [x] Setting up Conda environment to run this jupyter notebook

### Part 1 - Detecting a simulated ball


- Import the required python libraries

In [1]:
import numpy as np
import imutils
import cv2
import rospy
from cv_bridge import CvBridge
from sensor_msgs.msg import Image, CameraInfo
import time

ModuleNotFoundError: No module named 'sensor_msgs'

- [ ] First make sure you have roslaunched gazebo world runing in the background
- [x] Initialize a ros node by the name "ball_tracker". Follow the rospy [documentatiomn](http://wiki.ros.org/rospy/Overview/Initialization%20and%20Shutdown) to undestand the code line
- [x] Create an object of cv_bridge class. 
- [ ] You will be converting the image frames from RGB to HSV color space. Understand the [HSV](https://en.wikipedia.org/wiki/HSL_and_HSV) colour space interpretation. Define the threshold (Upper and Lower)for simulated ball in HSV color space. Read the OpenCV [documentation](https://docs.opencv.org/3.4.6/de/d25/imgproc_color_conversions.html#color_convert_rgb_hsv) to understand about OpenCV color converstion from RGB/BGR to HSV. 

In [ ]:
rospy.init_node('ball_tracker')
rate = rospy.Rate(0.5)
bridge = CvBridge()

## Your code begins here
Lower = (40, 40, 40)   # Enter relevant values as (Hue, Saturation,Value)
Upper = (70, 255, 255)
## Your code ends here

Complete the function to perfom following tasks
- [ ] Grab a ROS image by subscribing to the camera topic '/camera1/image_raw'. Hint: you can use [rospy.wait_for_message()](https://docs.ros.org/en/diamondback/api/rospy/html/rospy.client-pysrc.html#wait_for_message) to receive one message from a topic
- [ ] Convert the ROS image message to an OpenCV image (desired_encoding='rgb8') and return output using [CvBridge](http://wiki.ros.org/cv_bridge/Tutorials/ConvertingBetweenROSImagesAndOpenCVImagesPython)


In [3]:
def grab_frame():
    
    ## Your code begins here 
    img_message = rospy.wait_for_message('/camera1/image_raw', Image, timeout=5)
    frame = bridge.imgmsg_to_cv2(img_message, desired_encoding='rgb8')
    ## Your code ends here
    
    return frame

Create a function to perform the following process on image
- [ ] Resize the image with [imutils](https://www.pyimagesearch.com/2015/02/02/just-open-sourced-personal-imutils-package-series-opencv-convenience-functions/) while protecting the original aspect ratio. set image width to 600. 
- [ ]  Add gaussean blur to the image with OpenCV image [smoothing] (https://docs.opencv.org/4.5.2/d4/d13/tutorial_py_filtering.html)
-  [ ] Convert image from RGB to HSV using OpenCV image [conversion](https://docs.opencv.org/3.4/d8/d01/group__imgproc__color__conversions.html#gga4e0972be5de079fed4e3a10e24ef5ef0aa4a7f0ecf2e94150699e48c79139ee12)
-  [ ] Construct a mask for green color using OpenCV [thresholding](https://docs.opencv.org/3.4/da/d97/tutorial_threshold_inRange.html) functions
-  [ ] Perform Erosion and Dialation on the image mask using OpenCV [Morphological](https://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html) operationshttps://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html
-  [x] Return image mask and resized frame as output

In [5]:
def prep(frame, lower, upper):
    # Your code begins here
    # resize to 600x600
    frame = imutils.resize(frame, width = 600)
    # apply gaussian blur
    frame_blur = cv2.GaussianBlur(frame, (5,5), cv2.BORDER_DEFAULT)
    # rgb 2 hsv
    frame_HSV = cv2.cvtColor(frame_blur, cv2.COLOR_RGB2HSV)
    # thresholding
    mask = cv2.inRange(frame_HSV, lower, upper)
    # erosion & dilation
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=1)
    #Your code ends here
    return mask, frame

-  [ ] Create a function to find contours in the mask image and return a tuple consisting of contour coordinates using OpenCV [contour](https://docs.opencv.org/4.5.0/d4/d73/tutorial_py_contours_begin.html) functiuons

In [6]:
def find_cnts(mask):
    ## Your code begins here 
    # find contour in mask
    cnts = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    ## Your code ends here

    cnts = imutils.grab_contours(cnts)
    return cnts


-  [x] Assuming all the above functions are ciompleted, run the following code to view 3 orignal image frmae, image mask and tracking in 3 individual windows
-  [x] Press "Q" to abort

In [7]:
while not rospy.is_shutdown():
    
    frame = grab_frame()
    mask, frame  = prep(frame, Lower, Upper)
    
    # Display the frame grabbed from ROS message
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)    
   
    cnts  = find_cnts(mask) 
    

    center = None
    # proceed only if a contour is detected
    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        # Find center of contour using moments in opencvq
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        if radius > 10:
            cv2.circle(frame, (int(x), int(y)), int(radius),
                (0, 0, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)

    cv2.imshow("Tracking", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
cv2.destroyAllWindows()   

### Part 2 - Detecting a coloured blob from a video


-  [ ]  Now lets do the tracking of a blob in video file without using ROS. Have a look at the video file provided with the assignment folder. Initialize the new threshold values according to the color you want to detect.

In [7]:
# Your code begins here
Lower = (100, 50, 40)
Upper = (140, 255, 255)
# Your code ends here

In [8]:
blueI = cv2.imread("blue_test.jpg")
blueI_hsv = cv2.cvtColor(blueI, cv2.COLOR_BGR2HSV)
blueI_hsv
mask = cv2.inRange(blueI_hsv, Lower, Upper)
cv2.imshow('original', blueI)
cv2.imshow('mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()
# mask

In [9]:
def prep_video_frame(frame, lower, upper):
    # Your code begins here
    # resize to 600x600
    # frame = imutils.resize(frame, width = 600)
    # apply gaussian blur
    # frame_blur = cv2.GaussianBlur(frame, (5,5), cv2.BORDER_DEFAULT)
    # rgb 2 hsv
    frame_HSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # thresholding
    mask = cv2.inRange(frame_HSV, lower, upper)
    # erosion & dilation
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=1)
    #Your code ends here
    return mask, frame

In [10]:
def find_cnts(mask):
    ## Your code begins here 
    # find contour in mask
    cnts = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    ## Your code ends here

    cnts = imutils.grab_contours(cnts)
    return cnts


-  [ ] Complete the rest of the program to grab a frame from video clip and apply image processing functions to detect the color blob. You may reuse the functions from Part 1. You may also refer OpenCV documentation and implement new functions if you want.

In [11]:
# Your code here
vid = cv2.VideoCapture("./test_video.avi")

while vid.isOpened():
    # capture video frame by frame
    ret, frame = vid.read()
    if ret == True:
        
        time.sleep(0.05)
        mask, frame  = prep_video_frame(frame, Lower, Upper)

        cv2.imshow('frame', frame)
        cv2.imshow('Mask', mask)
        # press q to quit
        cnts  = find_cnts(mask) 
        

        center = None
        # proceed only if a contour is detected
        if len(cnts) > 0:
            c = max(cnts, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            # Find center of contour using moments in opencvq
            M = cv2.moments(c)
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            if radius > 10:
                cv2.circle(frame, (int(x), int(y)), int(radius),
                    (0, 0, 255), 2)
                cv2.circle(frame, center, 5, (0, 0, 255), -1)

        cv2.imshow("Tracking", frame)
    else:
        vid.set(cv2.CAP_PROP_POS_FRAMES, 0)  # restart video

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()